In [11]:
import numpy as np
import edward as ed
import tensorflow as tf

In [23]:
df = np.loadtxt('/home/john/Dev/edward/examples/data/crabs_train.txt',
                dtype='float32', delimiter=',')
df[df[:, 0] == -1, 0] = 0  # replace -1 label with 0 label

N = 25  # number of data points
D = df.shape[1] - 1  # number of features

subset = np.random.choice(df.shape[0], N, replace=False)
X_train = df[subset, 1:]
y_train = df[subset, 0]
y_train

array([ 1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.], dtype=float32)

In [7]:
from edward.models import Bernoulli, Normal, MultivariateNormalFull
from edward.util import multivariate_rbf

def kernel(x):
  mat = []
  for i in range(N):
    mat += [[]]
    xi = x[i, :]
    for j in range(N):
      if j == i:
        mat[i] += [multivariate_rbf(xi, xi)]
      else:
        xj = x[j, :]
        mat[i] += [multivariate_rbf(xi, xj)]

    mat[i] = tf.stack(mat[i])

  return tf.stack(mat)

X = tf.placeholder(tf.float32, [N, D])
f = MultivariateNormalFull(mu=tf.zeros(N), sigma=kernel(X))
y = Bernoulli(logits=f)

In [9]:
qf = Normal(mu=tf.Variable(tf.random_normal([N])),
            sigma=tf.nn.softplus(tf.Variable(tf.random_normal([N]))))

In [12]:
inference = ed.KLqp({f: qf}, data={X: X_train, y: y_train})
inference.run(n_iter=500)

Iteration   1 [  0%]: Loss = 62.834
Iteration  50 [ 10%]: Loss = 15.193
Iteration 100 [ 20%]: Loss = 22.526
Iteration 150 [ 30%]: Loss = 20.692
Iteration 200 [ 40%]: Loss = 25.175
Iteration 250 [ 50%]: Loss = 19.563
Iteration 300 [ 60%]: Loss = 23.187
Iteration 350 [ 70%]: Loss = 17.450
Iteration 400 [ 80%]: Loss = 20.063
Iteration 450 [ 90%]: Loss = 19.953
Iteration 500 [100%]: Loss = 18.338
